# Heaps & Priority Queues

*Efficient access to minimum or maximum elements*


## 🎯 Learning Objectives

- Understand the heap property (min-heap and max-heap) and how heaps are stored in arrays
- Implement heap operations: insert (sift-up), extract-min/max (sift-down), and heapify
- Build a heap from an array in O(n) time using bottom-up heapify
- Use Python's heapq module effectively for priority queue operations
- Apply heaps to solve top-K problems, running median, merge-K-sorted-lists, and scheduling problems

---
## 1. Introduction to Heaps


In the previous week, we learned about binary search trees that provide O(log n) operations but can degrade to O(n) without balancing. This week, we introduce **heaps**—a specialized tree structure that guarantees efficient access to the minimum or maximum element without the complexity of maintaining perfect balance.

> 📖 **Definition:** A **heap** is a complete binary tree that satisfies the **heap property**: in a **min-heap**, every parent is smaller than or equal to its children; in a **max-heap**, every parent is greater than or equal to its children. The root always contains the minimum (or maximum) element.

> 💡 **Min-Heap vs Max-Heap**
>
> ```

Min-Heap (parent ≤ children):        Max-Heap (parent ≥ children):

         1                                    9
       /   \                                /   \
      3     2                              7     8
     / \   /                              / \   /
    7   4 5                              3   6 5

Root = minimum (1)                  Root = maximum (9)
Useful for: get min, priority       Useful for: get max, priority
queues with smallest first          queues with largest first
                
```

### Why Heaps?

| Operation | Sorted Array | Unsorted Array | Heap |
| --- | --- | --- | --- |
| Find Min/Max | O(1) | O(n) | O(1) |
| Insert | O(n) | O(1) | O(log n) |
| Extract Min/Max | O(1) or O(n) | O(n) | O(log n) |
| Build from n elements | O(n log n) | O(n) | O(n) |

**Listing 7.1 — Heap vs Other Structures**

In [ ]:
import heapq
import time

n = 10000
data = list(range(n, 0, -1))  # Worst case: reverse sorted

# Approach 1: Keep sorted array (insert + sort)
sorted_arr = []
start = time.perf_counter()
for x in data[:1000]:  # Only 1000 for speed
    sorted_arr.append(x)
    sorted_arr.sort()  # O(n log n) each time!
sorted_time = time.perf_counter() - start

# Approach 2: Use heap
heap = []
start = time.perf_counter()
for x in data[:1000]:
    heapq.heappush(heap, x)  # O(log n) each time
heap_time = time.perf_counter() - start

print(f"Insert 1000 elements while maintaining order:")
print(f"  Sorted array: {sorted_time*1000:.2f} ms")
print(f"  Heap:         {heap_time*1000:.2f} ms")
speedup = sorted_time/heap_time if heap_time > 0 else float('inf')
print(f"  Speedup:      {speedup:.1f}x")

print(f"\nBoth give minimum: sorted[0]={sorted_arr[0]}, heap[0]={heap[0]}")

***Figure 7.1:** Heaps provide the best trade-off when you need repeated access to the minimum/maximum with insertions.*

---
## 2. Heap Properties


A heap has two essential properties:

> 📖 **Property 1: Complete Binary Tree:** A heap is a **complete binary tree**: all levels are fully filled except possibly the last level, which is filled from left to right. This ensures the tree is as compact as possible and can be efficiently stored in an array.

> 📖 **Property 2: Heap Order:** For a **min-heap**, every node's value is less than or equal to its children's values. For a **max-heap**, every node's value is greater than or equal to its children's values. Note: siblings have no ordering relationship.

> 💡 **Complete vs Incomplete Binary Tree**
>
> ```

Complete (valid heap shape):      Not Complete (invalid):

        1                              1
       / \                            / \
      2   3                          2   3
     / \                              \
    4   5                              4

All levels filled left-to-right   Gap at 2's left child
Can be stored as: [1,2,3,4,5]     Cannot use array efficiently
                
```

**Listing 7.2 — Verifying Heap Properties**

In [ ]:
def is_min_heap(arr):
    """Check if array represents a valid min-heap."""
    n = len(arr)
    for i in range(n):
        left = 2 * i + 1
        right = 2 * i + 2
        
        # Check left child
        if left < n and arr[i] > arr[left]:
            return False
        # Check right child
        if right < n and arr[i] > arr[right]:
            return False
    return True

def is_max_heap(arr):
    """Check if array represents a valid max-heap."""
    n = len(arr)
    for i in range(n):
        left = 2 * i + 1
        right = 2 * i + 2
        
        if left < n and arr[i] < arr[left]:
            return False
        if right < n and arr[i] < arr[right]:
            return False
    return True

# Test cases
min_heap = [1, 3, 2, 7, 4, 5]
max_heap = [9, 7, 8, 3, 6, 5]
not_heap = [1, 5, 2, 3, 4]  # 5 > parent's children rule violated

print(f"[1, 3, 2, 7, 4, 5] is min-heap: {is_min_heap(min_heap)}")
print(f"[9, 7, 8, 3, 6, 5] is max-heap: {is_max_heap(max_heap)}")
print(f"[1, 5, 2, 3, 4] is min-heap: {is_min_heap(not_heap)}")

# Note: min_heap is NOT a max_heap and vice versa
print(f"\n[1, 3, 2, 7, 4, 5] is max-heap: {is_max_heap(min_heap)}")

***Figure 7.2:** Heap property check verifies parent-child relationships for all non-leaf nodes.*

---
## 3. Array Representation


The complete binary tree property allows heaps to be efficiently stored in arrays without explicit pointers:

> 💡 **Tree to Array Mapping**
>
> ```

Tree representation:              Array representation:

           1 (i=0)                Index: 0  1  2  3  4  5
          / \                     Value: 1  3  2  7  4  5
         /   \
    (i=1) 3   2 (i=2)             For node at index i:
       / \   /                    • Parent:      (i-1) // 2
      /   \ /                     • Left child:  2*i + 1
 (i=3)7  4 5(i=5)                 • Right child: 2*i + 2
     (i=4)
                
```

**Listing 7.3 — Array Index Navigation**

In [ ]:
def parent(i):
    """Return parent index."""
    return (i - 1) // 2

def left_child(i):
    """Return left child index."""
    return 2 * i + 1

def right_child(i):
    """Return right child index."""
    return 2 * i + 2

def has_parent(i):
    return i > 0

def has_left_child(i, heap_size):
    return left_child(i) < heap_size

def has_right_child(i, heap_size):
    return right_child(i) < heap_size

# Demonstrate with heap [1, 3, 2, 7, 4, 5]
heap = [1, 3, 2, 7, 4, 5]
print(f"Heap: {heap}\n")

for i in range(len(heap)):
    info = f"Index {i} (value={heap[i]}): "
    
    if has_parent(i):
        info += f"parent={heap[parent(i)]} "
    else:
        info += "parent=None (root) "
    
    if has_left_child(i, len(heap)):
        info += f"left={heap[left_child(i)]} "
    else:
        info += "left=None "
        
    if has_right_child(i, len(heap)):
        info += f"right={heap[right_child(i)]}"
    else:
        info += "right=None"
    
    print(info)

***Figure 7.3:** Array indices provide O(1) navigation between parent and children without pointers.*

**Listing 7.4 — Visualizing Heap as Tree**

In [ ]:
def print_heap_tree(heap):
    """Print heap as a tree structure."""
    if not heap:
        print("Empty heap")
        return
    
    n = len(heap)
    height = 0
    while (1 << height) - 1 < n:
        height += 1
    
    max_width = (1 << height) * 2
    
    level = 0
    i = 0
    while i < n:
        level_size = 1 << level
        spacing = max_width // (level_size + 1)
        
        line = ""
        for j in range(level_size):
            if i + j < n:
                line += f"{heap[i + j]:^{spacing}}"
            else:
                break
        
        print(line)
        i += level_size
        level += 1

# Test with different heaps
print("Min-heap [1, 3, 2, 7, 4, 5, 6]:")
print_heap_tree([1, 3, 2, 7, 4, 5, 6])

print("\nMax-heap [9, 7, 8, 3, 6, 5, 2]:")
print_heap_tree([9, 7, 8, 3, 6, 5, 2])

***Figure 7.4:** Heap array can be visualized as a complete binary tree with level-order layout.*

---
## 4. Heap Operations


### Operation 1: Peek (Get Min/Max)

**Listing 7.5 — Peek Operation**

In [ ]:
class MinHeap:
    def __init__(self):
        self.heap = []
    
    def peek(self):
        """Return minimum without removing - O(1)."""
        if not self.heap:
            raise IndexError("Heap is empty")
        return self.heap[0]
    
    def __len__(self):
        return len(self.heap)
    
    def is_empty(self):
        return len(self.heap) == 0

# Test
heap = MinHeap()
heap.heap = [1, 3, 2, 7, 4, 5]  # Pre-built heap

print(f"Heap: {heap.heap}")
print(f"Peek (minimum): {heap.peek()}")
print(f"Heap unchanged: {heap.heap}")
print(f"Size: {len(heap)}")

***Figure 7.5:** Peek is O(1)—the minimum/maximum is always at index 0.*

### Operation 2: Insert (Push)

**Listing 7.6 — Insert with Bubble Up**

In [ ]:
class MinHeap:
    def __init__(self):
        self.heap = []
    
    def _parent(self, i):
        return (i - 1) // 2
    
    def _bubble_up(self, i):
        """Move element up until heap property restored."""
        while i > 0:
            parent_idx = self._parent(i)
            if self.heap[i] < self.heap[parent_idx]:
                # Swap with parent
                self.heap[i], self.heap[parent_idx] = \
                    self.heap[parent_idx], self.heap[i]
                i = parent_idx
            else:
                break
    
    def insert(self, value):
        """Add element to heap - O(log n)."""
        self.heap.append(value)  # Add at end
        self._bubble_up(len(self.heap) - 1)  # Restore heap
    
    def __repr__(self):
        return f"MinHeap({self.heap})"

# Demonstrate insertion
heap = MinHeap()
values = [5, 3, 8, 1, 2, 9, 4]

print("Inserting values:")
for v in values:
    heap.insert(v)
    print(f"  Insert {v}: {heap.heap}")

print(f"\nFinal heap: {heap}")
print(f"Minimum: {heap.heap[0]}")

***Figure 7.6:** Insert adds at the end, then bubbles up to restore heap property. O(log n) worst case.*

### Operation 3: Extract Min/Max

**Listing 7.7 — Extract with Bubble Down**

In [ ]:
class MinHeap:
    def __init__(self):
        self.heap = []
    
    def _left(self, i):
        return 2 * i + 1
    
    def _right(self, i):
        return 2 * i + 2
    
    def _bubble_down(self, i):
        """Move element down until heap property restored."""
        n = len(self.heap)
        while True:
            smallest = i
            left = self._left(i)
            right = self._right(i)
            
            if left < n and self.heap[left] < self.heap[smallest]:
                smallest = left
            if right < n and self.heap[right] < self.heap[smallest]:
                smallest = right
            
            if smallest != i:
                self.heap[i], self.heap[smallest] = \
                    self.heap[smallest], self.heap[i]
                i = smallest
            else:
                break
    
    def extract_min(self):
        """Remove and return minimum - O(log n)."""
        if not self.heap:
            raise IndexError("Heap is empty")
        
        min_val = self.heap[0]
        
        # Move last element to root
        self.heap[0] = self.heap[-1]
        self.heap.pop()
        
        # Restore heap property
        if self.heap:
            self._bubble_down(0)
        
        return min_val
    
    def insert(self, value):
        self.heap.append(value)
        i = len(self.heap) - 1
        while i > 0:
            parent = (i - 1) // 2
            if self.heap[i] < self.heap[parent]:
                self.heap[i], self.heap[parent] = self.heap[parent], self.heap[i]
                i = parent
            else:
                break

# Demonstrate extraction
heap = MinHeap()
for v in [1, 3, 2, 7, 4, 5]:
    heap.insert(v)

print(f"Initial heap: {heap.heap}")
print("\nExtracting in order:")
while heap.heap:
    val = heap.extract_min()
    print(f"  Extracted {val}, remaining: {heap.heap}")

***Figure 7.7:** Extract replaces root with last element, then bubbles down. Repeated extraction gives sorted order!*

---
## 5. Heapify Process


The **heapify** operation (also called sift-down or percolate-down) fixes a single violation of the heap property. It assumes both subtrees are valid heaps, but the root may violate the property.

**Listing 7.8 — Heapify (Sift Down)**

In [ ]:
def heapify_min(arr, n, i):
    """
    Heapify subtree rooted at index i.
    Assumes children are already valid heaps.
    Time: O(log n), Space: O(1)
    """
    smallest = i
    left = 2 * i + 1
    right = 2 * i + 2
    
    # Find smallest among root, left, right
    if left < n and arr[left] < arr[smallest]:
        smallest = left
    if right < n and arr[right] < arr[smallest]:
        smallest = right
    
    # If root is not smallest, swap and continue
    if smallest != i:
        arr[i], arr[smallest] = arr[smallest], arr[i]
        heapify_min(arr, n, smallest)  # Recurse on affected subtree

def heapify_max(arr, n, i):
    """Heapify for max-heap."""
    largest = i
    left = 2 * i + 1
    right = 2 * i + 2
    
    if left < n and arr[left] > arr[largest]:
        largest = left
    if right < n and arr[right] > arr[largest]:
        largest = right
    
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify_max(arr, n, largest)

# Example: fix a single violation
arr = [10, 3, 2, 7, 4, 5]  # 10 violates min-heap at root
print(f"Before heapify: {arr}")
print("  10 at root is larger than children 3 and 2")

heapify_min(arr, len(arr), 0)
print(f"After heapify:  {arr}")
print("  2 bubbled up to root, 10 moved down")

***Figure 7.8:** Heapify fixes one node by comparing with children and swapping with the smaller (min-heap) or larger (max-heap) child.*

**Listing 7.9 — Iterative Heapify**

In [ ]:
def heapify_iterative(arr, n, i):
    """Iterative heapify - O(1) space."""
    while True:
        smallest = i
        left = 2 * i + 1
        right = 2 * i + 2
        
        if left < n and arr[left] < arr[smallest]:
            smallest = left
        if right < n and arr[right] < arr[smallest]:
            smallest = right
        
        if smallest == i:
            break
        
        arr[i], arr[smallest] = arr[smallest], arr[i]
        i = smallest  # Continue from swapped position

# Compare recursive vs iterative
arr1 = [10, 3, 2, 7, 4, 5]
arr2 = [10, 3, 2, 7, 4, 5]

heapify_min(arr1, len(arr1), 0)  # Recursive
heapify_iterative(arr2, len(arr2), 0)  # Iterative

print(f"Recursive result: {arr1}")
print(f"Iterative result: {arr2}")
print(f"Same result: {arr1 == arr2}")

***Figure 7.9:** Iterative heapify avoids recursion overhead and uses O(1) extra space.*

---
## 6. Building a Heap


Given an array, we can build a heap in-place. There are two approaches:

### Approach 1: Repeated Insertion - O(n log n)

**Listing 7.10 — Build Heap via Insertion**

In [ ]:
def build_heap_insert(arr):
    """Build heap by inserting elements one by one - O(n log n)."""
    heap = []
    
    for val in arr:
        # Insert (append + bubble up)
        heap.append(val)
        i = len(heap) - 1
        
        while i > 0:
            parent = (i - 1) // 2
            if heap[i] < heap[parent]:
                heap[i], heap[parent] = heap[parent], heap[i]
                i = parent
            else:
                break
    
    return heap

arr = [4, 10, 3, 5, 1, 8, 7]
print(f"Original: {arr}")
heap = build_heap_insert(arr)
print(f"Heap (via insertion): {heap}")

# Verify it's a valid min-heap
def is_min_heap(arr):
    for i in range(len(arr)):
        left, right = 2*i+1, 2*i+2
        if left < len(arr) and arr[i] > arr[left]:
            return False
        if right < len(arr) and arr[i] > arr[right]:
            return False
    return True

print(f"Valid min-heap: {is_min_heap(heap)}")

***Figure 7.10:** Repeated insertion is intuitive but takes O(n log n) total time.*

### Approach 2: Bottom-Up Heapify - O(n)

**Listing 7.11 — Build Heap via Heapify**

In [ ]:
def heapify(arr, n, i):
    """Min-heapify subtree rooted at i."""
    smallest = i
    left = 2 * i + 1
    right = 2 * i + 2
    
    if left < n and arr[left] < arr[smallest]:
        smallest = left
    if right < n and arr[right] < arr[smallest]:
        smallest = right
    
    if smallest != i:
        arr[i], arr[smallest] = arr[smallest], arr[i]
        heapify(arr, n, smallest)

def build_heap_heapify(arr):
    """
    Build heap in-place using bottom-up heapify - O(n).
    Start from last non-leaf and heapify each node.
    """
    n = len(arr)
    # Last non-leaf is at index (n // 2) - 1
    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)
    return arr

arr = [4, 10, 3, 5, 1, 8, 7]
print(f"Original: {arr}")
build_heap_heapify(arr)
print(f"Heap (via heapify): {arr}")

# Why O(n)? Most nodes are near leaves and need few swaps
# Nodes at height h: n/2^(h+1), work per node: O(h)
# Total: sum of h * n/2^(h+1) = O(n)

***Figure 7.11:** Bottom-up heapify is O(n) because most nodes are near leaves and need minimal work.*

**Listing 7.12 — Build Heap Performance**

In [ ]:
import time
import random
import heapq

def build_via_insert(arr):
    heap = []
    for val in arr:
        heap.append(val)
        i = len(heap) - 1
        while i > 0:
            p = (i - 1) // 2
            if heap[i] < heap[p]:
                heap[i], heap[p] = heap[p], heap[i]
                i = p
            else:
                break
    return heap

def build_via_heapify(arr):
    arr = arr.copy()
    heapq.heapify(arr)  # Uses bottom-up heapify
    return arr

# Compare performance
sizes = [1000, 5000, 10000]

print("Build heap performance comparison:")
print("-" * 50)

for n in sizes:
    arr = [random.randint(1, 10000) for _ in range(n)]
    
    start = time.perf_counter()
    build_via_insert(arr.copy())
    insert_time = time.perf_counter() - start
    
    start = time.perf_counter()
    build_via_heapify(arr)
    heapify_time = time.perf_counter() - start
    
    print(f"n={n:5d}: insert={insert_time*1000:.2f}ms, "
          f"heapify={heapify_time*1000:.2f}ms, "
          f"ratio={insert_time/heapify_time:.1f}x")

***Figure 7.12:** Bottom-up heapify is significantly faster than repeated insertion for building heaps.*

---
## 7. Priority Queues


> 📖 **Definition:** A **priority queue** is an abstract data type where each element has a priority. Elements are dequeued in priority order, not insertion order. Heaps are the most common implementation of priority queues.

**Listing 7.13 — Priority Queue Implementation**

In [ ]:
class PriorityQueue:
    """Min-priority queue using heap."""
    
    def __init__(self):
        self._heap = []
    
    def push(self, item, priority):
        """Add item with given priority."""
        # Store as (priority, item) tuple
        entry = (priority, item)
        self._heap.append(entry)
        self._bubble_up(len(self._heap) - 1)
    
    def pop(self):
        """Remove and return lowest priority item."""
        if not self._heap:
            raise IndexError("Priority queue is empty")
        
        min_entry = self._heap[0]
        self._heap[0] = self._heap[-1]
        self._heap.pop()
        
        if self._heap:
            self._bubble_down(0)
        
        return min_entry[1]  # Return item, not tuple
    
    def peek(self):
        """Return lowest priority item without removing."""
        if not self._heap:
            raise IndexError("Priority queue is empty")
        return self._heap[0][1]
    
    def _bubble_up(self, i):
        while i > 0:
            parent = (i - 1) // 2
            if self._heap[i][0] < self._heap[parent][0]:
                self._heap[i], self._heap[parent] = \
                    self._heap[parent], self._heap[i]
                i = parent
            else:
                break
    
    def _bubble_down(self, i):
        n = len(self._heap)
        while True:
            smallest = i
            left, right = 2*i+1, 2*i+2
            if left < n and self._heap[left][0] < self._heap[smallest][0]:
                smallest = left
            if right < n and self._heap[right][0] < self._heap[smallest][0]:
                smallest = right
            if smallest != i:
                self._heap[i], self._heap[smallest] = \
                    self._heap[smallest], self._heap[i]
                i = smallest
            else:
                break
    
    def __len__(self):
        return len(self._heap)

# Test priority queue
pq = PriorityQueue()
tasks = [
    ("Low priority task", 3),
    ("Critical task", 1),
    ("Medium task", 2),
    ("Another low", 3),
]

print("Adding tasks:")
for task, priority in tasks:
    pq.push(task, priority)
    print(f"  Added '{task}' with priority {priority}")

print("\nProcessing in priority order:")
while len(pq) > 0:
    task = pq.pop()
    print(f"  Processing: {task}")

***Figure 7.13:** Priority queue wraps heap with meaningful interface. Lower priority value = higher urgency.*

**Listing 7.14 — Max-Priority Queue**

In [ ]:
import heapq

class MaxPriorityQueue:
    """Max-priority queue using negated priorities."""
    
    def __init__(self):
        self._heap = []
    
    def push(self, item, priority):
        """Add item - higher priority served first."""
        # Negate priority for max-heap behavior
        heapq.heappush(self._heap, (-priority, item))
    
    def pop(self):
        """Remove highest priority item."""
        if not self._heap:
            raise IndexError("Empty queue")
        return heapq.heappop(self._heap)[1]
    
    def peek(self):
        if not self._heap:
            raise IndexError("Empty queue")
        return self._heap[0][1]
    
    def __len__(self):
        return len(self._heap)

# Test max-priority queue
pq = MaxPriorityQueue()

print("Hospital ER Triage (higher = more urgent):")
patients = [
    ("Minor cut", 1),
    ("Heart attack", 5),
    ("Broken arm", 3),
    ("Severe allergic reaction", 4),
]

for patient, urgency in patients:
    pq.push(patient, urgency)
    print(f"  Admitted: {patient} (urgency {urgency})")

print("\nTreatment order:")
while len(pq) > 0:
    print(f"  Treating: {pq.pop()}")

***Figure 7.14:** Max-priority queue negates priorities to use Python's min-heap. Higher values processed first.*

---
## 8. Python's heapq Module


Python's `heapq` module provides heap operations on regular lists. It implements a min-heap.

**Listing 7.15 — heapq Basics**

In [ ]:
import heapq

# Create empty heap (just a list)
heap = []

# Push elements - O(log n) each
heapq.heappush(heap, 5)
heapq.heappush(heap, 3)
heapq.heappush(heap, 8)
heapq.heappush(heap, 1)
print(f"After pushes: {heap}")

# Peek at minimum - O(1)
print(f"Minimum: {heap[0]}")

# Pop minimum - O(log n)
min_val = heapq.heappop(heap)
print(f"Popped: {min_val}")
print(f"After pop: {heap}")

# Build heap from existing list - O(n)
data = [9, 5, 2, 7, 3, 8, 1]
print(f"\nOriginal list: {data}")
heapq.heapify(data)
print(f"After heapify: {data}")

# Push and pop in one operation - O(log n)
result = heapq.heappushpop(data, 4)  # Push 4, pop min
print(f"heappushpop(4): popped {result}, heap={data}")

# Pop and push - O(log n)
result = heapq.heapreplace(data, 0)  # Pop min, push 0
print(f"heapreplace(0): popped {result}, heap={data}")

***Figure 7.15:** heapq provides efficient heap operations. Remember: it's a min-heap operating on a regular list.*

**Listing 7.16 — heapq Advanced Functions**

In [ ]:
import heapq

# nlargest and nsmallest - efficient for small k
data = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
print(f"Data: {data}")
print(f"3 smallest: {heapq.nsmallest(3, data)}")
print(f"3 largest:  {heapq.nlargest(3, data)}")

# With key function
words = ['apple', 'pie', 'banana', 'cherry', 'date']
print(f"\nWords: {words}")
print(f"2 shortest: {heapq.nsmallest(2, words, key=len)}")
print(f"2 longest:  {heapq.nlargest(2, words, key=len)}")

# Merge sorted iterables
list1 = [1, 3, 5, 7]
list2 = [2, 4, 6, 8]
list3 = [0, 9]
print(f"\nMerging sorted lists: {list1}, {list2}, {list3}")
merged = list(heapq.merge(list1, list2, list3))
print(f"Result: {merged}")

# When to use nlargest/nsmallest vs sorted:
# - k << n: use nlargest/nsmallest O(n + k log n)
# - k ≈ n: use sorted() O(n log n)
# - k = 1: use min()/max() O(n)

***Figure 7.16:** heapq also provides nlargest, nsmallest, and merge for common operations.*

**Listing 7.17 — Max-Heap with heapq**

In [ ]:
import heapq

# heapq only provides min-heap
# For max-heap, negate values

class MaxHeap:
    def __init__(self):
        self._heap = []
    
    def push(self, val):
        heapq.heappush(self._heap, -val)
    
    def pop(self):
        return -heapq.heappop(self._heap)
    
    def peek(self):
        return -self._heap[0]
    
    def __len__(self):
        return len(self._heap)

# Test
max_heap = MaxHeap()
values = [3, 1, 4, 1, 5, 9, 2, 6]

print("Building max-heap:")
for v in values:
    max_heap.push(v)
    print(f"  Push {v}: max={max_heap.peek()}")

print("\nExtracting in descending order:")
while len(max_heap) > 0:
    print(f"  Pop: {max_heap.pop()}")

***Figure 7.17:** Negate values to simulate max-heap with Python's min-heap implementation.*

**Listing 7.18 — Custom Objects in Heap**

In [ ]:
import heapq

# For custom objects, use tuples (priority, tiebreaker, object)
# Tiebreaker ensures comparison works even with equal priorities

class Task:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f"Task({self.name})"

# Using tuple approach
task_heap = []
counter = 0  # Tiebreaker for equal priorities

tasks = [
    (2, "Write code"),
    (1, "Fix critical bug"),
    (2, "Review PR"),
    (3, "Update docs"),
]

for priority, name in tasks:
    task = Task(name)
    # (priority, counter, task) - counter breaks ties
    heapq.heappush(task_heap, (priority, counter, task))
    counter += 1

print("Tasks in priority order:")
while task_heap:
    priority, _, task = heapq.heappop(task_heap)
    print(f"  Priority {priority}: {task.name}")

***Figure 7.18:** Use tuples with a counter as tiebreaker when heap elements might have equal priorities.*

---
## 9. Heap Applications


### Application 1: Heap Sort

**Listing 7.19 — Heap Sort**

In [ ]:
def heapify(arr, n, i):
    """Max-heapify for ascending sort."""
    largest = i
    left = 2 * i + 1
    right = 2 * i + 2
    
    if left < n and arr[left] > arr[largest]:
        largest = left
    if right < n and arr[right] > arr[largest]:
        largest = right
    
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest)

def heap_sort(arr):
    """
    Sort array using heap - O(n log n) time, O(1) space.
    1. Build max-heap
    2. Repeatedly extract max to end
    """
    n = len(arr)
    
    # Build max-heap
    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)
    
    # Extract elements one by one
    for i in range(n - 1, 0, -1):
        arr[0], arr[i] = arr[i], arr[0]  # Move max to end
        heapify(arr, i, 0)  # Heapify reduced heap

arr = [12, 11, 13, 5, 6, 7]
print(f"Original: {arr}")
heap_sort(arr)
print(f"Sorted:   {arr}")

***Figure 7.19:** Heap sort builds max-heap, then repeatedly moves max to sorted portion. O(n log n) and in-place.*

### Application 2: K Largest/Smallest Elements

**Listing 7.20 — K Largest Elements**

In [ ]:
import heapq

def k_largest_sort(arr, k):
    """Sort and take last k - O(n log n)."""
    return sorted(arr)[-k:]

def k_largest_heap(arr, k):
    """
    Use min-heap of size k - O(n log k).
    Keep k largest seen so far.
    """
    heap = []
    for num in arr:
        if len(heap) < k:
            heapq.heappush(heap, num)
        elif num > heap[0]:
            heapq.heapreplace(heap, num)  # Pop smallest, push new
    return sorted(heap, reverse=True)

def k_largest_builtin(arr, k):
    """Use heapq.nlargest - optimized implementation."""
    return heapq.nlargest(k, arr)

# Test
arr = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5, 8, 9, 7]
k = 4

print(f"Array: {arr}")
print(f"k = {k}")
print(f"\nk largest (sort):    {k_largest_sort(arr, k)}")
print(f"k largest (heap):    {k_largest_heap(arr, k)}")
print(f"k largest (builtin): {k_largest_builtin(arr, k)}")

***Figure 7.20:** For k largest, use min-heap of size k. Each element is O(log k), total O(n log k).*

### Application 3: Merge K Sorted Lists

**Listing 7.21 — Merge K Sorted Lists**

In [ ]:
import heapq

def merge_k_sorted(lists):
    """
    Merge k sorted lists into one sorted list.
    Time: O(n log k) where n is total elements
    """
    result = []
    heap = []  # (value, list_index, element_index)
    
    # Initialize with first element of each list
    for i, lst in enumerate(lists):
        if lst:
            heapq.heappush(heap, (lst[0], i, 0))
    
    while heap:
        val, list_idx, elem_idx = heapq.heappop(heap)
        result.append(val)
        
        # If there's a next element in that list, add it
        if elem_idx + 1 < len(lists[list_idx]):
            next_val = lists[list_idx][elem_idx + 1]
            heapq.heappush(heap, (next_val, list_idx, elem_idx + 1))
    
    return result

# Test
lists = [
    [1, 4, 7, 10],
    [2, 5, 8, 11],
    [3, 6, 9, 12],
]

print("Sorted lists:")
for i, lst in enumerate(lists):
    print(f"  List {i}: {lst}")

merged = merge_k_sorted(lists)
print(f"\nMerged: {merged}")

***Figure 7.21:** Keep one element from each list in heap. Always pop smallest, push its successor.*

### Application 4: Running Median

**Listing 7.22 — Running Median with Two Heaps**

In [ ]:
import heapq

class MedianFinder:
    """
    Find median in O(log n) per insert, O(1) per query.
    Use two heaps: max-heap for lower half, min-heap for upper half.
    """
    
    def __init__(self):
        self.low = []   # Max-heap (negated) for smaller half
        self.high = []  # Min-heap for larger half
    
    def add(self, num):
        # Add to max-heap of lower half
        heapq.heappush(self.low, -num)
        
        # Ensure max of low <= min of high
        if self.high and -self.low[0] > self.high[0]:
            val = -heapq.heappop(self.low)
            heapq.heappush(self.high, val)
        
        # Balance sizes: low can have at most 1 more than high
        if len(self.low) > len(self.high) + 1:
            val = -heapq.heappop(self.low)
            heapq.heappush(self.high, val)
        elif len(self.high) > len(self.low):
            val = heapq.heappop(self.high)
            heapq.heappush(self.low, -val)
    
    def median(self):
        if len(self.low) > len(self.high):
            return -self.low[0]
        return (-self.low[0] + self.high[0]) / 2

# Test
mf = MedianFinder()
numbers = [2, 3, 4, 1, 5, 6]

print("Running median:")
for num in numbers:
    mf.add(num)
    print(f"  Add {num}: median = {mf.median()}")

***Figure 7.22:** Two heaps split data at median. Max-heap holds smaller half, min-heap holds larger half.*

### Application 5: Top K Frequent Elements

**Listing 7.23 — Top K Frequent**

In [ ]:
import heapq
from collections import Counter

def top_k_frequent(nums, k):
    """
    Find k most frequent elements.
    Time: O(n log k)
    """
    # Count frequencies - O(n)
    count = Counter(nums)
    
    # Use min-heap of size k
    # Heap contains (frequency, element)
    heap = []
    for num, freq in count.items():
        if len(heap) < k:
            heapq.heappush(heap, (freq, num))
        elif freq > heap[0][0]:
            heapq.heapreplace(heap, (freq, num))
    
    return [num for freq, num in heap]

# Test
nums = [1, 1, 1, 2, 2, 3, 3, 3, 3, 4]
k = 2

print(f"Array: {nums}")
print(f"Frequencies: {dict(Counter(nums))}")
print(f"Top {k} frequent: {top_k_frequent(nums, k)}")

***Figure 7.23:** Count frequencies, then use size-k heap to keep the k most frequent.*

### Application 6: Dijkstra's Shortest Path

**Listing 7.24 — Dijkstra with Heap**

In [ ]:
import heapq

def dijkstra(graph, start):
    """
    Find shortest paths from start to all nodes.
    Time: O((V + E) log V) with heap
    """
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    
    # Priority queue: (distance, node)
    pq = [(0, start)]
    
    while pq:
        dist, node = heapq.heappop(pq)
        
        # Skip if we found a better path already
        if dist > distances[node]:
            continue
        
        for neighbor, weight in graph[node].items():
            new_dist = dist + weight
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                heapq.heappush(pq, (new_dist, neighbor))
    
    return distances

# Example graph
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1},
}

print("Graph:")
for node, edges in graph.items():
    print(f"  {node}: {edges}")

distances = dijkstra(graph, 'A')
print(f"\nShortest distances from A:")
for node, dist in sorted(distances.items()):
    print(f"  A -> {node}: {dist}")

***Figure 7.24:** Dijkstra's algorithm uses a min-heap to always process the nearest unvisited node.*

---
## 10. Common Patterns


### Pattern 1: Kth Element Problems

**Listing 7.25 — Kth Largest in Stream**

In [ ]:
import heapq

class KthLargest:
    """
    Find kth largest element in a stream.
    Maintain min-heap of size k.
    """
    
    def __init__(self, k, nums):
        self.k = k
        self.heap = []
        for num in nums:
            self.add(num)
    
    def add(self, val):
        if len(self.heap) < self.k:
            heapq.heappush(self.heap, val)
        elif val > self.heap[0]:
            heapq.heapreplace(self.heap, val)
        return self.heap[0]  # kth largest

# Test
k = 3
initial = [4, 5, 8, 2]
kth = KthLargest(k, initial)

print(f"Initial: {initial}, k={k}")
print(f"Heap (k largest): {sorted(kth.heap, reverse=True)}")
print(f"Kth largest: {kth.heap[0]}")

for val in [3, 5, 10, 9, 4]:
    result = kth.add(val)
    print(f"Add {val}: kth largest = {result}")

***Figure 7.25:** For kth largest in stream, keep min-heap of size k. Root is always the kth largest.*

### Pattern 2: Scheduling Problems

**Listing 7.26 — Task Scheduler**

In [ ]:
import heapq
from collections import Counter

def least_interval(tasks, n):
    """
    Find minimum time to complete all tasks.
    Same task must have n intervals between executions.
    """
    # Count task frequencies
    freq = list(Counter(tasks).values())
    
    # Max-heap of frequencies (negate for max-heap)
    heap = [-f for f in freq]
    heapq.heapify(heap)
    
    time = 0
    while heap:
        temp = []
        
        # Try to fill n+1 slots in this cycle
        for _ in range(n + 1):
            if heap:
                count = -heapq.heappop(heap)
                if count > 1:
                    temp.append(-(count - 1))
            time += 1
            
            # If no more tasks, stop
            if not heap and not temp:
                break
        
        # Add remaining tasks back
        for item in temp:
            heapq.heappush(heap, item)
    
    return time

# Test
tasks = ['A', 'A', 'A', 'B', 'B', 'B']
n = 2

print(f"Tasks: {tasks}")
print(f"Cooling interval: {n}")
print(f"Minimum time: {least_interval(tasks, n)}")
# A -> B -> idle -> A -> B -> idle -> A -> B = 8

***Figure 7.26:** Greedy scheduling: always process most frequent task first to minimize idle time.*

### Pattern 3: Meeting Rooms

**Listing 7.27 — Minimum Meeting Rooms**

In [ ]:
import heapq

def min_meeting_rooms(intervals):
    """
    Find minimum number of meeting rooms required.
    Heap tracks end times of ongoing meetings.
    """
    if not intervals:
        return 0
    
    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    
    # Heap of end times (rooms in use)
    rooms = []
    heapq.heappush(rooms, intervals[0][1])
    
    for start, end in intervals[1:]:
        # If earliest ending room is free, reuse it
        if start >= rooms[0]:
            heapq.heappop(rooms)
        
        # Add current meeting's end time
        heapq.heappush(rooms, end)
    
    return len(rooms)

# Test
meetings = [(0, 30), (5, 10), (15, 20)]
print(f"Meetings: {meetings}")
print(f"Rooms needed: {min_meeting_rooms(meetings)}")

meetings2 = [(7, 10), (2, 4)]
print(f"\nMeetings: {meetings2}")
print(f"Rooms needed: {min_meeting_rooms(meetings2)}")

***Figure 7.27:** Track meeting end times in heap. When new meeting starts, check if any room is free.*

### Pattern 4: Sliding Window Maximum

**Listing 7.28 — Max in Sliding Window**

In [ ]:
import heapq

def max_sliding_window(nums, k):
    """
    Find maximum in each sliding window of size k.
    Using max-heap with lazy removal.
    """
    result = []
    # Max-heap: (-value, index)
    heap = []
    
    for i, num in enumerate(nums):
        # Add current element
        heapq.heappush(heap, (-num, i))
        
        # Remove elements outside window
        while heap[0][1] <= i - k:
            heapq.heappop(heap)
        
        # Add max to result once window is full
        if i >= k - 1:
            result.append(-heap[0][0])
    
    return result

# Test
nums = [1, 3, -1, -3, 5, 3, 6, 7]
k = 3

print(f"Array: {nums}, k={k}")
print(f"Max in each window: {max_sliding_window(nums, k)}")

# Verify manually
print("\nWindows:")
for i in range(len(nums) - k + 1):
    window = nums[i:i+k]
    print(f"  {window} -> max = {max(window)}")

***Figure 7.28:** Max-heap with lazy removal of out-of-window elements. Check index before using top.*

### Pattern 5: Skyline Problem

**Listing 7.29 — Skyline Outline**

In [ ]:
import heapq

def get_skyline(buildings):
    """
    Get skyline silhouette from buildings.
    Each building: [left, right, height]
    """
    # Create events: (x, type, height)
    # Start: negative height (process taller first)
    # End: positive height
    events = []
    for left, right, height in buildings:
        events.append((left, -height, right))  # Start
        events.append((right, 0, 0))  # End marker
    
    events.sort()
    
    result = []
    # Max-heap of (negative height, end position)
    # Use (0, float('inf')) as ground
    heap = [(0, float('inf'))]
    
    for x, neg_h, end in events:
        if neg_h != 0:  # Start of building
            heapq.heappush(heap, (neg_h, end))
        
        # Remove buildings that have ended
        while heap[0][1] <= x:
            heapq.heappop(heap)
        
        # Current max height
        max_h = -heap[0][0]
        
        # Add to result if height changed
        if not result or result[-1][1] != max_h:
            result.append([x, max_h])
    
    return result

# Test
buildings = [[2, 9, 10], [3, 7, 15], [5, 12, 12]]
print(f"Buildings: {buildings}")
skyline = get_skyline(buildings)
print(f"Skyline: {skyline}")

***Figure 7.29:** Skyline uses sweep line with max-heap to track tallest active building at each position.*

### Pattern 6: IPO (Project Selection)

**Listing 7.30 — Maximum Capital**

In [ ]:
import heapq

def find_maximized_capital(k, w, profits, capital):
    """
    Select at most k projects to maximize capital.
    Each project needs minimum capital to start.
    w = starting capital
    """
    n = len(profits)
    # Projects sorted by capital required
    projects = sorted(zip(capital, profits))
    
    available = []  # Max-heap of profits (negated)
    i = 0
    
    for _ in range(k):
        # Add all affordable projects to heap
        while i < n and projects[i][0] <= w:
            heapq.heappush(available, -projects[i][1])
            i += 1
        
        # If no projects available, stop
        if not available:
            break
        
        # Do the most profitable project
        w += -heapq.heappop(available)
    
    return w

# Test
k = 2
w = 0
profits = [1, 2, 3]
capital = [0, 1, 1]

print(f"Starting capital: {w}")
print(f"Projects (capital, profit): {list(zip(capital, profits))}")
print(f"Select at most {k} projects")
print(f"Maximum capital: {find_maximized_capital(k, w, profits, capital)}")

***Figure 7.30:** Greedy selection: always pick most profitable affordable project. Two heaps pattern.*

- **Heap** is a complete binary tree with heap property (parent vs children)
- **Min-heap:** root is minimum; **Max-heap:** root is maximum
- **Array storage:** parent at (i-1)//2, children at 2i+1 and 2i+2
- **Operations:** peek O(1), insert O(log n), extract O(log n)
- **Build heap:** use heapify for O(n) instead of repeated insert O(n log n)
- **Python:** use `heapq` module (min-heap only, negate for max)

---
## 11. Common Pitfalls


### Pitfall 1: Confusing heapify with heappush

**Listing 7.31 — heapify vs heappush**

In [ ]:
import heapq
import time

data = list(range(10000, 0, -1))  # Reverse sorted

# WRONG: Using heappush in loop - O(n log n)
heap1 = []
start = time.perf_counter()
for x in data:
    heapq.heappush(heap1, x)
push_time = time.perf_counter() - start

# RIGHT: Using heapify - O(n)
heap2 = data.copy()
start = time.perf_counter()
heapq.heapify(heap2)
heapify_time = time.perf_counter() - start

print(f"Building heap from {len(data)} elements:")
print(f"  heappush loop: {push_time*1000:.2f} ms")
print(f"  heapify:       {heapify_time*1000:.2f} ms")
speedup = push_time/heapify_time if heapify_time > 0 else float('inf')
print(f"  Speedup:       {speedup:.1f}x")
print(f"\nBoth valid: {heap1[0] == heap2[0] == 1}")

***Figure 7.31:** Always use heapify() to build a heap from existing data, not repeated heappush().*

### Pitfall 2: Forgetting heapq is Min-Heap Only

**Listing 7.32 — Max-Heap Patterns**

In [ ]:
import heapq

# WRONG: Expecting max-heap behavior
data = [3, 1, 4, 1, 5, 9]
heapq.heapify(data)
print(f"heapq gives minimum: {data[0]}")  # 1, not 9!

# RIGHT: Negate for max-heap
max_heap = [-x for x in [3, 1, 4, 1, 5, 9]]
heapq.heapify(max_heap)
print(f"Negated gives maximum: {-max_heap[0]}")  # 9

# For objects, use negative priority
tasks = [(3, 'low'), (1, 'high'), (2, 'medium')]

# Max-priority (higher number = higher priority)
max_pq = [(-p, t) for p, t in tasks]
heapq.heapify(max_pq)

print("\nMax-priority queue order:")
while max_pq:
    neg_p, task = heapq.heappop(max_pq)
    print(f"  Priority {-neg_p}: {task}")

***Figure 7.32:** Python's heapq is always a min-heap. Negate values for max-heap behavior.*

### Pitfall 3: Modifying Heap Elements

**Listing 7.33 — Safe Heap Updates**

In [ ]:
import heapq

# WRONG: Modifying element in heap
heap = [[5, 'task1'], [3, 'task2'], [8, 'task3']]
heapq.heapify(heap)
print(f"Before: {heap}")
print(f"Min: {heap[0]}")

# DON'T DO THIS - breaks heap property!
heap[1][0] = 1  # Changing priority
print(f"After modification: {heap}")
print(f"Min is wrong: {heap[0]} (should be [1, 'task3'])")

# RIGHT: Remove and re-add, or use lazy deletion
# Option 1: Rebuild heap
heapq.heapify(heap)
print(f"After re-heapify: {heap}")

# Option 2: Mark entries as invalid (lazy deletion)
# See pattern with (priority, counter, item) tuples

***Figure 7.33:** Never modify heap elements directly. Re-heapify or use lazy deletion pattern.*

---
# 📝 Exercises


### Exercise 1: Last Stone Weight  (⭐ Easy)

We have stones with weights. Each turn, smash two heaviest stones together. If weights are equal, both destroyed. If not, the lighter is destroyed and the heavier's weight becomes the difference. Return the weight of the last remaining stone (or 0 if none).

**Expected:** (Expected: [2,7,4,1,8,1] → 1)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Max-Heap:**

                        Use a max-heap to always get the two heaviest stones. Negate values since heapq is min-heap.
- **Hint 2 - Smash Logic:**

                        Pop two largest. If different, push the difference back onto the heap.
- **Hint 3 - Termination:**

                        Continue until 0 or 1 stones remain. Return remaining stone weight or 0.
</details>

In [ ]:
# ✏️ [EX1]
# Last Stone Weight - Smash heaviest stones

import heapq

def last_stone_weight(stones):
    # Your code here
    pass

# Test your implementation (uncomment)
# stones = [2, 7, 4, 1, 8, 1]
# print(f"Stones: {stones}")
# print(f"Last stone weight: {last_stone_weight(stones)}")  # Expected: 1

### Exercise 2: K Closest Points to Origin  (⭐⭐ Medium)

Given an array of points where points[i] = [xi, yi], return the k closest points to the origin (0, 0). Distance is Euclidean.

**Expected:** (Expected: Return k points with smallest distance)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Distance Formula:**

                        Distance² = x² + y² (no need for sqrt when comparing).
- **Hint 2 - Max-Heap of Size k:**

                        Use max-heap (negate distance) to track k closest points.
- **Hint 3 - Maintain k Closest:**

                        If heap size > k, pop the farthest point.
</details>

In [ ]:
# ✏️ [EX2]
# K Closest Points to Origin

import heapq

def k_closest(points, k):
    # Your code here
    pass

# Test your implementation (uncomment)
# points = [[1, 3], [-2, 2], [5, 8], [0, 1]]
# k = 2
# print(f"Points: {points}")
# print(f"{k} closest to origin: {k_closest(points, k)}")

### Exercise 3: Reorganize String  (⭐⭐ Medium)

Given a string s, rearrange so that no two adjacent characters are the same. Return any valid arrangement, or "" if impossible.

**Expected:** (Expected: "aab" → "aba", "aaab" → "")

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Impossibility Check:**

                        If any character appears more than `(len(s)+1)//2` times, it's impossible.
- **Hint 2 - Max-Heap:**

                        Use a max-heap of (count, char) to always pick the most frequent available character.
- **Hint 3 - Alternate Characters:**

                        After placing a character, hold it aside until the next character is placed.
</details>

In [ ]:
# ✏️ [EX3]
# Reorganize String - No adjacent duplicates

import heapq
from collections import Counter

def reorganize_string(s):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(f"'aab' -> '{reorganize_string('aab')}'")  # "aba"
# print(f"'aaab' -> '{reorganize_string('aaab')}'")  # ""

### Exercise 4: Find Median from Data Stream  (⭐⭐⭐ Hard)

Implement MedianFinder class with addNum(int num) and findMedian() methods. findMedian should return the median of all elements added so far.

**Expected:** (Expected: Two heaps - max for lower half, min for upper half)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Two Heaps:**

                        Max-heap for smaller half, min-heap for larger half.
- **Hint 2 - Balance Heaps:**

                        Keep heaps balanced (differ by at most 1 element).
- **Hint 3 - Find Median:**

                        Median is top of larger heap, or average of both tops if equal size.
</details>

In [ ]:
# ✏️ [EX4]
# Find Median from Data Stream

import heapq

class MedianFinder:
    def __init__(self):
        # Your code here
        pass
    
    def addNum(self, num):
        # Your code here
        pass
    
    def findMedian(self):
        # Your code here
        pass

# Test your implementation (uncomment)
# mf = MedianFinder()
# mf.addNum(1); print(f"Added 1, median: {mf.findMedian()}")
# mf.addNum(2); print(f"Added 2, median: {mf.findMedian()}")
# mf.addNum(3); print(f"Added 3, median: {mf.findMedian()}")

### Exercise 5: Ugly Number II  (⭐⭐⭐ Hard)

An ugly number is a positive integer whose prime factors are limited to 2, 3, and 5. Given n, return the nth ugly number. (1 is considered ugly.)

**Expected:** (Expected: n=10 → 12, n=15 → 24)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Start with 1:**

                        Initialize heap with 1 (first ugly number).
- **Hint 2 - Generate Next:**

                        Pop minimum, multiply by 2, 3, 5, add results to heap.
- **Hint 3 - Avoid Duplicates:**

                        Use a set to track seen numbers and avoid duplicates.
</details>

In [ ]:
# ✏️ [EX5]
# Ugly Number II - Prime factors limited to 2, 3, 5

import heapq

def nth_ugly_number(n):
    # Your code here
    pass

# Test your implementation (uncomment)
# for i in [1, 10, 15]:
#     print(f"Ugly number {i}: {nth_ugly_number(i)}")
# First 15: 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,10}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-10 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request

WEEK = "Week_07"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ── Check info was filled in ──
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Run the cell above first to set your info!")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Run the cell above first — some fields are empty.")

# ── Extract exercise answers from IPython history ──
_answers = {}
try:
    _ipy = get_ipython()
    _hist = _ipy.history_manager.get_range(output=False)
    for _sess, _line, _src in _hist:
        _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
        if _m:
            _ex_id = "ex" + _m.group(1)
            _lines = _src.split("\n")
            _clean = "\n".join(_lines[1:]).strip()
            _answers[_ex_id] = {
                "code": _clean,
                "modified": len(_clean) > 5
            }
except Exception:
    pass

# ── Fallback: also check In[] from current session ──
if not _answers:
    try:
        for _src in In:
            if not _src:
                continue
            _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
            if _m:
                _ex_id = "ex" + _m.group(1)
                _lines = _src.split("\n")
                _clean = "\n".join(_lines[1:]).strip()
                _answers[_ex_id] = {
                    "code": _clean,
                    "modified": len(_clean) > 5
                }
    except NameError:
        pass

# ── Fallback: try reading notebook file (VS Code) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir(".") if f.endswith(".ipynb") and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), "r", encoding="utf-8") as _f:
            _nb = json.load(_f)
        for _cell in _nb["cells"]:
            if _cell["cell_type"] != "code":
                continue
            _src = "".join(_cell["source"]) if isinstance(_cell["source"], list) else _cell["source"]
            _m = re.match(r"#\s*✏️\s*\[EX(\w+)\]", _src)
            if _m:
                _ex_id = "ex" + _m.group(1)
                _lines = _src.split("\n")
                _clean = "\n".join(_lines[1:]).strip()
                _answers[_ex_id] = {
                    "code": _clean,
                    "modified": len(_clean) > 5
                }

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys()))}")

if not _answers:
    print("\n⚠️  No exercise answers found!")
    print("Make sure you RAN all exercise cells before submitting.")
    raise SystemExit()

# ── Send ──
_data = json.dumps({
    "week": WEEK,
    "studentId": _sid,
    "studentName": _sname,
    "studentEmail": _semail,
    "classCode": _scode,
    "source": "dsa-notebook",
    "timeOnPage": 0,
    "answers": _answers
}).encode("utf-8")

print("📡 Submitting...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get("success"):
        print(f"\n✅ {_result['message']}")
        print("📧 Check your email for confirmation.")
    else:
        print(f"\n❌ {_result.get('message', 'Submission failed')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={"Content-Type": "text/plain"}, method="POST")
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  Request sent — check your email for confirmation.")
    print(f"(If no email arrives, try again or contact your instructor)")
